<a href="https://colab.research.google.com/github/Alvanatus/Simulacion-ll/blob/main/Serpientes%20y%20Escaleras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Para estimar el valor esperado de N, primero necesitamos entender las probabilidades asociadas con el juego.

Supongamos que en cada turno se lanza un dado de 6 caras, y sea 𝑝 la probabilidad de avanzar a la siguiente casilla. Dado que el dado tiene 6 caras y el juego tiene 20 casillas en total, la probabilidad de avanzar a la siguiente casilla es 𝑝 = 1/6. Sin embargo, hay algunas casillas especiales que afectan el movimiento:

Si el jugador cae en una casilla con una escalera, se mueve a la casilla superior de la escalera.
Si el jugador cae en una casilla con una serpiente, se mueve a la casilla inferior de la serpiente.

Para estimar el valor esperado de N, podemos usar un enfoque analítico y otro basado en simulación.

 **Enfoque analítico:**
Para calcular el valor esperado de N analíticamente, podemos usar conceptos de probabilidad. Sea E𝑖 la esperanza condicional del número de lanzamientos restantes si estamos en la casilla i. El valor esperado total E será la suma de estas esperanzas condicionales ponderadas por las probabilidades de llegar a cada casilla:

E = Σ𝑖=1→20 P(i) ⋅ E𝑖

Para calcular E𝑖, podemos usar programación dinámica o un enfoque recursivo. Comenzamos con E20 = 0 (porque estamos en la casilla final) y luego retrocedemos hasta E1, utilizando las probabilidades de movimiento y las casillas especiales.

**Enfoque por simulación:**
Para estimar el valor esperado de N por simulación, simplemente simulamos múltiples juegos y promediamos el número de lanzamientos requeridos para llegar a la casilla final.



# Enfoque por simulación.

1. Especificar las reglas del juego, incluyendo el tablero, las serpientes y escaleras, y cómo se mueve el jugador.

2. Escribir un programa que simule múltiples juegos. En cada juego, el programa lanzará un dado en cada turno y moverá al jugador según el resultado del dado y las reglas del juego.

3. Contar el número de lanzamientos requeridos para que el jugador alcance la casilla final en cada juego.

4. Calcular el promedio del número de lanzamientos requeridos para llegar a la casilla final sobre todas las simulaciones.




In [1]:
import random

def jugar_juego():
    # Definir el tablero con las serpientes y escaleras
    tablero = {
        3: 11, 15: 19,  # Escaleras
        4: 13, 10: 17   # Serpientes
    }

    # Posición inicial del jugador
    posicion = 1
    lanzamientos = 0

    # Mientras no lleguemos a la casilla final
    while posicion < 20:
        # Lanzar el dado
        dado = random.randint(1, 6)
        lanzamientos += 1

        # Mover al jugador
        posicion += dado

        # Verificar si la posición tiene una serpiente o escalera
        if posicion in tablero:
            posicion = tablero[posicion]

        # Asegurarse de no pasarse de la casilla final
        posicion = min(posicion, 20)

    return lanzamientos

def simulacion(num_simulaciones):
    total_lanzamientos = 0

    # Realizar las simulaciones
    for _ in range(num_simulaciones):
        lanzamientos = jugar_juego()
        total_lanzamientos += lanzamientos

    # Calcular el promedio
    promedio = total_lanzamientos / num_simulaciones
    return promedio

# Número de simulaciones
num_simulaciones = 10000

# Realizar la simulación y mostrar el resultado
resultado = simulacion(num_simulaciones)
print(f"El valor esperado de N, basado en {num_simulaciones} simulaciones, es aproximadamente {resultado:.2f} lanzamientos.")


El valor esperado de N, basado en 10000 simulaciones, es aproximadamente 4.47 lanzamientos.


#Programa calculo de matriz de transicion

In [2]:
import numpy as np

# Definir las casillas especiales (escaleras y serpientes)
escaleras = {(3, 11), (15, 19)}
serpientes = {(4, 13), (10, 17)}

# Crear la matriz de transición
def crear_matriz_transicion():
  """
  Crea la matriz de transición del juego.

  Argumentos:
    escaleras: Diccionario que representa las escaleras (casilla inicial, casilla final).
    serpientes: Diccionario que representa las serpientes (casilla inicial, casilla final).

  Retorno:
    Matriz de transición NumPy.
  """

  matriz_transicion = np.zeros((20, 20), dtype=float)

  # Probabilidades para casillas sin casillas especiales
  for i in range(1, 20):
    for j in range(1, 7):
      nueva_casilla = i + j
      if nueva_casilla <= 20:
        matriz_transicion[i - 1, nueva_casilla - 1] += 1/6

  # Probabilidades para casillas con escaleras
  for (casilla_inicial, casilla_final) in escaleras:
    matriz_transicion[casilla_inicial - 1, casilla_final - 1] = 1/6

  # Probabilidades para casillas con serpientes
  for (casilla_inicial, casilla_final) in serpientes:
    matriz_transicion[casilla_inicial - 1, casilla_final - 1] = 1/6

  # Ajustar probabilidades para casillas con escaleras y serpientes
  for i in range(1, 20):
    for j in range(1, 7):
      nueva_casilla = i + j
      if (i, nueva_casilla) in escaleras:
        matriz_transicion[i - 1, i - 1] -= 1/6
      elif (i, nueva_casilla) in serpientes:
        matriz_transicion[i - 1, i - 1] -= 1/6

  return matriz_transicion

# Calcular las probabilidades de los estados
def calcular_probabilidades_estados(matriz_transicion):
  """
  Calcula las probabilidades de los estados utilizando el método de las Cadenas de Markov.

  Argumentos:
    matriz_transicion: Matriz de transición NumPy.

  Retorno:
    Vector de probabilidades de los estados (NumPy array).
  """

  v_inicial = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  v_actual = v_inicial

  while True:
    v_siguiente = matriz_transicion @ v_actual
    diferencia = np.linalg.norm(v_siguiente - v_actual)

    if diferencia < 1e-6:
      break

    v_actual = v_siguiente

  return v_siguiente

# Ejecutar el programa
matriz_transicion = crear_matriz_transicion()
probabilidades_estados = calcular_probabilidades_estados(matriz_transicion)

print("Matriz de transición:")
print(matriz_transicion)

print("\nProbabilidades de los estados:")
print(probabilidades_estados)


Matriz de transición:
[[ 0.          0.16666667  0.16666667  0.16666667  0.16666667  0.16666667
   0.16666667  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.16666667  0.16666667  0.16666667  0.16666667
   0.16666667  0.16666667  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.16666667  0.16666667  0.16666667
   0.16666667  0.16666667  0.16666667  0.          0.16666667  0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.16666667  0.16666667
   0.16666667  0.16666667  0.16666667  0.16666667  0.          0.
   0.16666667  0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.     

#Programa simulador de juego y calculo de N

In [3]:
import random

# Definición de las casillas
casillas = {
    3: 11, 15: 19,  # Escaleras
    4: 13, 10: 17   # Serpientes
}

# Función para mover la ficha
def mover_ficha(posicion, dado):
    """
    Mueve la ficha por la casilla indicada por el dado.

    Args:
        posicion: Posición actual de la ficha.
        dado: Resultado del lanzamiento del dado.

    Returns:
        Nueva posición de la ficha.
    """
    nueva_posicion = posicion + dado
    if nueva_posicion > 20:
        nueva_posicion = 20  # Si excede el límite, retroceder a la casilla 20
    if nueva_posicion in casillas:
        nueva_posicion = casillas[nueva_posicion]
    return nueva_posicion

# Función para jugar
def jugar():
    """
    Simula una partida del juego Serpientes y Escaleras.
    """
    posicion = 1
    lanzamientos = 0
    print(f"Empiezas en la casilla 1.")
    while posicion < 20:
        dado = random.randint(1, 6)
        lanzamientos += 1
        nueva_posicion = mover_ficha(posicion, dado)
        print(f"Después del lanzamiento {lanzamientos}, estás en la casilla {nueva_posicion}.")
        if nueva_posicion == 20:
            print(f"¡Llegaste a la casilla 20 en {lanzamientos} lanzamientos!")
            break
        posicion = nueva_posicion

# Inicio del juego
jugar()


Empiezas en la casilla 1.
Después del lanzamiento 1, estás en la casilla 13.
Después del lanzamiento 2, estás en la casilla 14.
Después del lanzamiento 3, estás en la casilla 16.
Después del lanzamiento 4, estás en la casilla 19.
Después del lanzamiento 5, estás en la casilla 20.
¡Llegaste a la casilla 20 en 5 lanzamientos!
